# DATA 410 Lecture 9 - Spring 2022

<font face="Chalkboard" color="darkgreen" size=10> Multivariate Models for Regression</font>

## Multivariate Models

In general we want

$$ \mathbb{E}(y):=F(x_1,x_2,x_3,...x_p)$$

where $F$ represents the model (regressor) we consider.

### Variable Selection

- We want to select only the features that are really important for our model.

- If the functional input-output model is $Y = F(X_1,X_2,X_3,X_4,X_5...X_p)$ then we imagine that it is very possible that only a subset of the variables $X_1,X_2,X_3,X_4,X_5...X_p$ are important and we need to disconsider (eliminate from the model) those that are not relevant.

- Programming and algorithms are based on equations, functions and statement evaluations.

- To represent variable selection in a functional way, we can think of multiplying each variable from the model by a binary weight, a weight of $0$ means the feature is not important and a weight of $1$ means that it is important:

$$
Y = F(w_1\cdot X_1,w_2\cdot X_2,w_3\cdot X_3,w_4\cdot X_4,w_5\cdot X_5...w_p\cdot X_p)
$$

where the weights $w_i$ are either $0$ or $1.$

The vector of binary weights $w=(w_1,w_2,w_3,...w_p)$ gives us what we call the ***sparsity pattern*** for the variable selection.

### Critical Aspects

1. What is the simplest choice for the function $F$?
2. How do we perform variable selection?
3. How do we accomodate nonlinear relationships?

## Variable Selection

In the case of multiple linear regression we have that

$$F(X_1,X_2,...X_p)=\beta_1 X_1+\beta_2 X_2 + ...\beta_p X_p$$

and the sparsity pattern means that a subset of the $\beta_1, \beta_2, ...\beta_p$ are equal to $0.$ 

So we assume 

$$ Y \approx X\cdot \beta +\sigma\epsilon $$

and we want the coefficients $\beta.$ 

The "classical" way of solving is:

$$X^{t}\cdot Y \approx X^{t}X\cdot \beta + \sigma X^{t}\epsilon$$ so we get $$ \mathbb{E}(\beta) = (X^{t}X)^{-1} X^{t}\cdot \mathbb{E}(Y)$$

where $\mathbb{E}(Y)$ denotes the expected value of $Y.$

The questions that we explore are:

 - Why and how we know that we need variable selection.
 
 - How we measure the effects of variable selection on the model.
 
 - How to determine if the method of selecting a sparsity pattern is working in the context of our data. 




Let's assume that we have some data with three features, such as

  Housing Area  |     Value   |    Property Tax  | Sales Price |
 -------------  |    ------   |   -------------  | ----------- |
          1800  |       234   |             9.8  |       267.5
          1980  |       244   |            10.5  |       278.2
          2120  |       252   |            16.2  |       284.5
          2500  |       280   |            18.4  |       310.4


If this is the only data we have for these features do we need to do any variable selection for predicting the sales price with a linear model?



If we try to fit the *Ordinary Least Squares* model (OLS) to determine the best fit what would we do?

```r
X <- matrix(c(1800,1980,2120, 2500, 234,244,252,280,9.8,10.5,16.2,18.4),4,3)
Y <- c(267.5,278.2,284.5,310.4)
model <- lm(Y~X)
model
```

## Gradient Boosting

Assume you have an regressor $F$ and, for the observation $x_i$ we make the prediction $F(x_i)$. To improve the predictions, we can regard $F$ as a 'weak learner' and therefore train a decision tree (we can call it $h$) where the new output is $y_i-F(x_i)$. Thus, there are increased chances that the new regressor

$$\large F + h$$ 

is better than the old one, $F.$

In [5]:
import numpy as np
import pandas as pd
from scipy.linalg import lstsq
from scipy.sparse.linalg import lsmr
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split as tts
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.tree import DecisionTreeRegressor

# imports for creating a Neural Network
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam # they recently updated Tensorflow
from keras.callbacks import EarlyStopping

In [4]:
# Tricubic Kernel
def Tricubic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,70/81*(1-d**3)**3)

# Quartic Kernel
def Quartic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,15/16*(1-d**2)**2)

# Epanechnikov Kernel
def Epanechnikov(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,3/4*(1-d**2))

#Defining the kernel local regression model

def lw_reg(X, y, xnew, kern, tau, intercept):
    # tau is called bandwidth K((x-x[i])/(2*tau))
    n = len(X) # the number of observations
    yest = np.zeros(n)

    if len(y.shape)==1: # here we make column vectors
      y = y.reshape(-1,1)

    if len(X.shape)==1:
      X = X.reshape(-1,1)
    
    if intercept:
      X1 = np.column_stack([np.ones((len(X),1)),X])
    else:
      X1 = X

    w = np.array([kern((X - X[i])/(2*tau)) for i in range(n)]) # here we compute n vectors of weights

    #Looping through all X-points
    for i in range(n):          
        W = np.diag(w[:,i])
        b = np.transpose(X1).dot(W).dot(y)
        A = np.transpose(X1).dot(W).dot(X1)
        #A = A + 0.001*np.eye(X1.shape[1]) # if we want L2 regularization
        #theta = linalg.solve(A, b) # A*theta = b
        beta, res, rnk, s = lstsq(A, b)
        yest[i] = np.dot(X1[i],beta)
    if X.shape[1]==1:
      f = interp1d(X.flatten(),yest,fill_value='extrapolate')
    else:
      f = LinearNDInterpolator(X, yest)
    output = f(xnew) # the output may have NaN's where the data points from xnew are outside the convex hull of X
    if sum(np.isnan(output))>0:
      g = NearestNDInterpolator(X,y.ravel()) 
      # output[np.isnan(output)] = g(X[np.isnan(output)])
      output[np.isnan(output)] = g(xnew[np.isnan(output)])
    return output

### Creating a Gradient Boosted Version of LWR

In [8]:
def boosted_lwr(X,y,xtest,kern,tau,intercept):
    #we need decision trees. For training the boosted lwr we use X and y (X is the full data, used as an xtrain matrix)
    #then once we train, we apply the boosting algorithm to new data
    Fx = lw_reg(X,y,X,Tricubic,tau,intercept) #output of lwr, used for training the decision trees
    #Now train the tree on yi - Fx (difference between real and regressor predictions)
    new_y = y - Fx
    tree_model = DecisionTreeRegressor(max_depth=3, random_state=123) #other hyperparameters currently at default
    tree_model.fit(X,new_y)
    output = lw_reg(X,y,xtest,kern,tau,intercept) + tree_model.predict(xtest)
    return output
    

In [13]:
data = pd.read_csv("Data/cars.csv")
X = data[["CYL","ENG","WGT"]].values
y = data['MPG'].values

scaler = StandardScaler()

#KFold between to see how effective each is
mse_lw = []

model_nn = Sequential()
model_nn.add(Dense(64, activation="relu", input_dim=3))
model_nn.add(Dense(32, activation="relu"))
model_nn.add(Dense(8, activation="relu"))
model_nn.add(Dense(1, activation="linear"))

model_nn.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=1e-3, decay=1e-3 / 200))
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1000)

kf = KFold(n_splits=10,shuffle=True,random_state=1234)

for idx_train, idx_test in kf.split(X):
    xtrain = scaler.fit_transform(X[idx_train])
    xtest = scaler.fit_transform(X[idx_test])

    ytrain = y[idx_train]
    ytest = y[idx_test]

    yhat_lw = boosted_lwr(xtrain,ytrain,xtest,Tricubic,1,intercept=True)
    
    mse_lw.append(mse(ytest, yhat_lw))

    
print("The Cross-validated MSE for Boosted multi dimensional Loess was: " + str(np.mean(mse_lw)))

The Cross-validated MSE for Boosted multi dimensional Loess was: 20.669118972100534


In [ ]:
X = np.transpose(np.array([[1800,1980,2120, 2500], [234,244,252,280],[9.8,10.5,16.2,18.4]]))
Y = np.array([267.5,278.2,284.5,310.4]).reshape(-1,1)

In [ ]:
df = pd.DataFrame(data=X,columns=['Housing Area','Value','Property Tax'])

In [ ]:
np.linalg.det(np.matmul(np.transpose(X),X)) # we yay !!!

-0.022219667101180705

In [ ]:
lm  = LinearRegression()
lm.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
lm.coef_

array([[ 0.03193903,  0.52413576, -0.41483344]])

In [ ]:
# Very important: the predictions should be made only in the range of the data
lm.intercept_

array([91.42734129])

In [ ]:
lm.predict([[2000,240,12]])

array([[276.11998743]])


## Discussion about intercept (with Python code)

A simpler example (below) in Python demonstrates the intercept based on the "mtcars" data set where we can use the line of best fit (also known as *Ordinary Least Squares regression*) in order to predict the mileage (mpg) by using the weight feature (wt). The weights of cars are measured in tons.

```python
# Libraries of functions need to be imported
# Then we can import the data that was saved in the same folder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model

cars = pd.read_csv("mtcars.csv")

cars.head(5)

x = cars["wt"]
y = cars["mpg"]
lm = linear_model.LinearRegression()
model = lm.fit(x,y)

# the following are the slope and the y-intercept 
slope = lm.coef_
y_intercept = lm.intercept_

x_range = np.arange(7)

yhat  = lm.predict(x_range.reshape(-1,1))
yhatd = lm.predict(x)
dy = y.values - yhatd
fig, ax = plt.subplots()
ax.plot(x_range.reshape(-1,1), yhat, '-',color='red')
ax.scatter(cars["wt"].values,cars["mpg"].values,s=30,facecolors='none',edgecolors='blue')
ax.vlines(x.values,y.values,y.values+dy,lw=0.5)
ax.set_xlim(0, 6)
ax.set_ylim(0, 50)
ax.set_xlabel('Weight (1unit = 1000lbs)',fontsize=14)
ax.set_ylabel('Miles Per Gallon',fontsize=14)
ax.grid(b=True,which='major', color ='grey', linestyle='-', alpha=0.8)
ax.grid(b=True,which='minor', color ='grey', linestyle='--', alpha=0.2)
ax.minorticks_on()
plt.show()
```



OLS works by minimizing the sum of the squared residuals in order to determine the slope and the intercept:

$$
\text{minimize} \sum_{i=1}^{n}\left(y_i -n - m\cdot x_i\right)^2
$$

Evidently, this is the case of a univariate input (only one feature as input) and a univariate output and the predicted values are $\hat{y}_i=m\cdot x_i+n.$


How many *weights* (also known as coefficients) did we estimate in total? The answer is 4 (3 features plus an intercept).

- We care of having an intercept because the data was not standardized, such as the mean of each feature column is not $0.$

So why exactly we know that *OLS* was supposed to work?

- We compute the determinant of $X^TX$ and if this is different from $0$ we know that *OLS* can be applied and we get weights for our features.

```r
det(t(X)%*%X)
model
```


What would happen if we had more features in the data?

   Dist to School |    Property Area   |    Housing Area   |      Value   |    Property Tax |  Sales Price
------------------ | ----------------   |   -------------   |   --------   |   ------------- | ------------
               7.0 |              0.4   |           1800    |        234   |             9.8 |        267.5
               2.3 |              0.8   |           1980    |        244   |            10.5 |        278.2
               4.3 |              1.1   |           2120    |        252   |            16.2 |        284.5
               3.8 |              0.6   |           2500    |        280   |            18.4 |        310.4


Can we determine the weights by applying the *OLS* model ? Let's see..




```r
X <- matrix(c(7.0,2.3,4.3,3.8,0.4,0.8,1.1,0.6,
              1800,1980,2120, 2500, 234,244,252,280,9.8,10.5,16.2,18.4),4,5)
X
det(t(X)%*%X)
```




```{r, echo=TRUE}
model <- lm(Y ~ X)
model
```

<div class="green">
Critical Thinking: 

- What is the message of this paragraph?

- What will happen if we add more features but the number of observations remains the same?
</div>

## Variable Selection

It is pretty clear by now that we need to select at most three variables in order for our multiple linear regression to work by applying the least squares method.

How exactly can we choose variables and how many (model) choices do we have?

- If our approach is exhaustive, we have in total $2^5=32$ possible sparsity patterns. Imagine how many sparsity patterns we can get if the total number of features is $100.$

The main idea is that we do not want to and we do not have all the resources to analyze all possible sparsity patterns, therefore we need to impose some extra restrictions or conditions that will help us select one sparsity pattern or just get the weights for all features.

## Regularization

One way to achieve variable selection or simply determine reasonable values for all weights is the method *regularization*, which translates to an optimization problem with constraints on the vector of weights.

That is to say that we minimize the sum of squared residuals with a *constraint* imposed on the vector of weights:

$$
\text{minimize} \sum_{i=1}^{n}\left(y_i -\beta_0-\sum_{j=1}^{p}X_{ij}\beta_j \right)^2
$$

subject to a constraint on $(\beta_1,\beta_2,...\beta_p)$ such as, for example, $\sum_{j=0}^{p}|\beta_j|<C$ or $\sum_{j=0}^{p}\beta_j^2<C$

 Let's consider the following example in R:
 
 ```r
library(lmridge)
model <- lmridge(Y ~ ., as.data.frame(X), K=0.001, scaling=("scaled"))
model
```

What would happen if we had more features in the data?

   Dist to School |    Property Area   |    Housing Area   |      Value   |    Property Tax |  Sales Price
------------------ | ----------------   |   -------------   |   --------   |   ------------- | ------------
               7.0 |              0.4   |           1800    |        234   |             9.8 |        267.5
               2.3 |              0.8   |           1980    |        244   |            10.5 |        278.2
               4.3 |              1.1   |           2120    |        252   |            16.2 |        284.5
               3.8 |              0.6   |           2500    |        280   |            18.4 |        310.4


Can we determine the weights by applying the *OLS* model ? Let's see..




```r
X <- matrix(c(7.0,2.3,4.3,3.8,0.4,0.8,1.1,0.6,
              1800,1980,2120, 2500, 234,244,252,280,9.8,10.5,16.2,18.4),4,5)
X
det(t(X)%*%X)
```

In [ ]:
X = np.array([[7.0,2.3,4.3,3.8],[0.4,0.8,1.1,0.6],
              [1800,1980,2120, 2500], [234,244,252,280],[9.8,10.5,16.2,18.4]])

In [ ]:
X = np.transpose(X)

In [ ]:
X.shape # the claim is that we NEED variable selection or regularization

(4, 5)

In [ ]:
# the determinant of X' * X is 0
np.linalg.det(np.matmul(np.transpose(X),X))

-0.022219667101180705

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
lr = Ridge(alpha=0.01)
lr.fit(X,Y)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
# we got the regularized coeffiecients by Ridge regression
lr.coef_

array([[ 0.05499038, -0.0880771 ,  0.03861881,  0.4372389 , -0.47123514]])

In [ ]:
# This is important: update the statsmodels package
! pip install --upgrade Cython
! pip install --upgrade git+https://github.com/statsmodels/statsmodels
import statsmodels.api as sm

Requirement already up-to-date: Cython in /usr/local/lib/python3.7/dist-packages (0.29.22)
  Cloning https://github.com/statsmodels/statsmodels to /tmp/pip-req-build-px_04k_0
  Running command git clone -q https://github.com/statsmodels/statsmodels /tmp/pip-req-build-px_04k_0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for statsmodels: filename=statsmodels-0.13.0.dev0+266.g6d4d588b0-cp37-cp37m-linux_x86_64.whl size=17998708 sha256=f10762e8de3148393f924dc481f44596637bbe6e33920d4e6addf4416fba3f5d
  Stored in directory: /tmp/pip-ephem-wheel-cache-hg6ie4lf/wheels/7d/ad/45/ac1a03bd759c2fa74c486e2b1950d94b55f511b4c2b0418bd5
Successfully built statsmodels
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
# general imports
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [ ]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/Boston Housing Prices.csv')
df

,town,tract,longitude,latitude,crime,residential,industrial,river,nox,rooms,older,distance,highway,tax,ptratio,lstat,cmedv
0,Nahant,2011,-70.955002,42.255001,0.00632,18.0,2.31,no,0.538,6.575,65.199997,4.0900,1,296,15.300000,4.98,24.000000
1,Swampscott,2021,-70.949997,42.287498,0.02731,0.0,7.07,no,0.469,6.421,78.900002,4.9671,2,242,17.799999,9.14,21.600000
2,Swampscott,2022,-70.935997,42.283001,0.02729,0.0,7.07,no,0.469,7.185,61.099998,4.9671,2,242,17.799999,4.03,34.700001
3,Marblehead,2031,-70.928001,42.292999,0.03237,0.0,2.18,no,0.458,6.998,45.799999,6.0622,3,222,18.700001,2.94,33.400002
4,Marblehead,2032,-70.921997,42.298000,0.06905,0.0,2.18,no,0.458,7.147,54.200001,6.0622,3,222,18.700001,5.33,36.200001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Winthrop,1801,-70.986000,42.231201,0.06263,0.0,11.93,no,0.573,6.593,69.099998,2.4786,1,273,21.000000,9.67,22.400000
502,Winthrop,1802,-70.990997,42.227501,0.04527,0.0,11.93,no,0.573,6.120,76.699997,2.2875,1,273,21.000000,9.08,20.600000
503,Winthrop,1803,-70.994797,42.226002,0.06076,0.0,11.93,no,0.573,6.976,91.000000,2.1675,1,273,21.000000,5.64,23.900000
504,Winthrop,1804,-70.987503,42.223999,0.10959,0.0,11.93,no,0.573,6.794,89.300003,2.3889,1,273,21.000000,6.48,22.000000


Extract the features we want (specifying them without any criteria is not optimal)

In [ ]:
features = ['crime','rooms','residential','industrial','nox','older','distance','highway','tax','ptratio','lstat']
X = np.array(df[features])
y = np.array(df['cmedv']).reshape(-1,1)
dat = np.concatenate([X,y], axis=1)

In [ ]:
X.shape

(506, 11)

In [ ]:
from sklearn.model_selection import train_test_split as tts
dat_train, dat_test = tts(dat, test_size=0.3, random_state=1234)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(dat_train[:,:-1],dat_train[:,-1])
yhat_lm = lm.predict(dat_test[:,:-1])
mae_lm = mean_absolute_error(dat_test[:,-1], yhat_lm)
print("MAE Linear Model = ${:,.2f}".format(1000*mae_lm))

MAE Linear Model = $3,640.02


### Critical Thinking Question: Is this the best we can do with a linear model?

In [ ]:
dat_test

## Neural Network Approach

In [ ]:
# imports for creating a Neural Networks
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
dat_train[:,:-1].shape

(354, 11)

In [ ]:
# Create a Neural Network model
model = Sequential()
model.add(Dense(128, activation="relu", input_dim=11))
model.add(Dense(32, activation="relu"))
model.add(Dense(8, activation="relu"))
# Since the regression is performed, a Dense layer containing a single neuron with a linear activation function.
# Typically ReLu-based activation are used but since it is performed regression, it is needed a linear activation.
model.add(Dense(1, activation="linear"))

# Compile model: The model is initialized with the Adam optimizer and then it is compiled.
model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-3, decay=1e-3 / 200))

# Patient early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=800)

# Fit the model
history = model.fit(dat_train[:,:-1], dat_train[:,11], validation_split=0.3, epochs=1000, batch_size=100, verbose=0, callbacks=[es])

# Calculate predictions
#yhat_nn = model.predict(X_test)

## Here are the predictions we made for the test data:

In [ ]:
from sklearn.metrics import mean_absolute_error

yhat_nn = model.predict(dat_test[:,:-1])
mae_nn = mean_absolute_error(dat_test[:,-1], yhat_nn)
print("MAE Neural Network = ${:,.2f}".format(1000*mae_nn))

MAE Neural Network = $2,652.57


In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
ax.set_xlim(3, 9)
ax.set_ylim(0, 51)
ax.scatter(x=df['rooms'], y=df['cmedv'],s=25)
ax.plot(X_test, lm.predict(X_test), color='red',label='Linear Regression')
ax.plot(dat_test[:,0], yhat_nn, color='lightgreen',lw=2.5,label='Neural Network')
ax.plot(dat_test[:,0], model_lowess(dat_train,dat_test,Epanechnikov,0.53), color='orange',lw=2.5,label='Kernel Weighted Regression')
ax.set_xlabel('Number of Rooms',fontsize=16,color='navy')
ax.set_ylabel('House Price (Thousands of Dollars)',fontsize=16,color='navy')
ax.set_title('Boston Housing Prices',fontsize=16,color='purple')
ax.grid(b=True,which='major', color ='grey', linestyle='-', alpha=0.8)
ax.grid(b=True,which='minor', color ='grey', linestyle='--', alpha=0.2)
ax.minorticks_on()
plt.legend()

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1693)

In [ ]:
mae_lm = []

for idxtrain, idxtest in kf.split(dat):
  X_train = dat[idxtrain,0]
  y_train = dat[idxtrain,1]
  X_test  = dat[idxtest,0]
  y_test = dat[idxtest,1]
  lm.fit(X_train.reshape(-1,1),y_train)
  yhat_lm = lm.predict(X_test.reshape(-1,1))
  mae_lm.append(mean_absolute_error(y_test, yhat_lm))
print("Validated MAE Linear Regression = ${:,.2f}".format(1000*np.mean(mae_lm)))

Validated MAE Linear Regression = $4,447.94


In [ ]:
%%timeit -n 1

mae_lk = []

for idxtrain, idxtest in kf.split(dat):
  dat_test = dat[idxtest,:]
  y_test = dat_test[np.argsort(dat_test[:, 0]),1]
  yhat_lk = model_lowess(dat[idxtrain,:],dat[idxtest,:],Gaussian,0.15)
  mae_lk.append(mean_absolute_error(y_test, yhat_lk))
print("Validated MAE Local Kernel Regression = ${:,.2f}".format(1000*np.mean(mae_lk)))

Validated MAE Local Kernel Regression = $4,090.03
Validated MAE Local Kernel Regression = $4,090.03
Validated MAE Local Kernel Regression = $4,090.03
1 loop, best of 3: 531 ms per loop


In [ ]:
#%%timeit -n 1

mae_nn = []

for idxtrain, idxtest in kf.split(dat):
  X_train = dat[idxtrain,0]
  y_train = dat[idxtrain,1]
  X_test  = dat[idxtest,0]
  y_test = dat[idxtest,1]
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=500)
  model.fit(X_train.reshape(-1,1),y_train,validation_split=0.3, epochs=1000, batch_size=100, verbose=0, callbacks=[es])
  yhat_nn = model.predict(X_test.reshape(-1,1))
  mae_nn.append(mean_absolute_error(y_test, yhat_nn))
print("Validated MAE Neural Network Regression = ${:,.2f}".format(1000*np.mean(mae_nn)))

Epoch 00501: early stopping
Epoch 00506: early stopping
Epoch 00539: early stopping
Epoch 00800: early stopping
Epoch 00672: early stopping
Epoch 00689: early stopping
Epoch 00551: early stopping
Validated MAE Neural Network Regression = $5,108.02


## XGBoost

The method is related to Random Forest

https://towardsdatascience.com/xgboost-python-example-42777d01001e

In [ ]:
import xgboost as xgb

In [ ]:
model_xgb = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=3)

In [ ]:
%%timeit -n 1

mae_xgb = []

for idxtrain, idxtest in kf.split(dat):
  X_train = dat[idxtrain,0]
  y_train = dat[idxtrain,1]
  X_test  = dat[idxtest,0]
  y_test = dat[idxtest,1]
  model_xgb.fit(X_train.reshape(-1,1),y_train)
  yhat_xgb = model_xgb.predict(X_test.reshape(-1,1))
  mae_xgb.append(mean_absolute_error(y_test, yhat_xgb))
print("Validated MAE XGBoost Regression = ${:,.2f}".format(1000*np.mean(mae_xgb)))

Validated MAE XGBoost Regression = $4,179.17
Validated MAE XGBoost Regression = $4,179.17
Validated MAE XGBoost Regression = $4,179.17
1 loop, best of 3: 181 ms per loop


In [ ]:
cstring = 'c'*364

## Using Kernel Regression from StatsModels

In [ ]:
hello = 'c'*dat_train_poly.shape[1]

In [ ]:
hello

'ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc'

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=2)
dat_train_poly = poly.fit_transform(dat_train)

In [ ]:
from statsmodels.nonparametric.kernel_regression import KernelReg


model_KernReg = KernelReg(endog=dat_train[:,-1],exog=dat_train_poly,var_type=hello,ckertype='gaussian')

In [ ]:
# Implementation of stepwise regression
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details """
    
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
df_train = pd.DataFrame(data=dat_train[:,:-1])

In [ ]:
stepwise_selection(df_train,dat_train[:,-1])

Add                              10 with p-value 2.95846e-61
Add                               1 with p-value 5.47201e-14
Add                               9 with p-value 2.50685e-11
Add                               6 with p-value 0.000788183
Add                               4 with p-value 8.34391e-08
Add                               2 with p-value 0.00268086


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: Deprecation

[10, 1, 9, 6, 4, 2]

In [ ]:
dat_train[:,-1].reshape(-1,1).shape

(354, 1)

In [ ]:
yhat_sm_test, y_std = model_KernReg.fit(dat_test[:,:-1])

In [ ]:
mae_sm = mean_absolute_error(dat_test[:,-1], yhat_sm_test)
print("MAE StatsModels Kernel Regression = ${:,.2f}".format(1000*mae_sm))

MAE StatsModels Kernel Regression = $2,854.57
